In [1]:
import glob


In [2]:
def get_nb_of_truth_probes(filename):
    nb_of_truth_probes = 0
    with open(filename) as fh:
        for line in fh:
            if line.startswith(">"):
                nb_of_truth_probes += 1
    return nb_of_truth_probes

truth_probes_files = glob.glob("/home/leandro/git/pandora1_paper/snippy_debug_2/hps/nobackup/research/zi/leandro/pandora1_paper/analysis_output_new_recall_pandora_nanopore_100x_best_snippy_precision_flank_150_recall_flank_50/recall/truth_probesets/**/*.fa")

samples_to_nb_of_truth_probes={}
for truth_probes_file in truth_probes_files:
    sample_where_truth_probe_comes_from = truth_probes_file.split("/")[-2]
    comparison = truth_probes_file.split("/")[-1].split(".")[0]
    samples_to_nb_of_truth_probes[(sample_where_truth_probe_comes_from, comparison)] = get_nb_of_truth_probes(truth_probes_file)

samples_to_nb_of_truth_probes

{('063_STEC', '063_STEC_and_CFT073'): 40251,
 ('063_STEC', '063_STEC_and_H131800734'): 99096,
 ('063_STEC', '063_STEC_and_ST38'): 103640,
 ('H131800734', 'H131800734_and_ST38'): 90014,
 ('H131800734', '063_STEC_and_H131800734'): 99096,
 ('H131800734', 'CFT073_and_H131800734'): 99509,
 ('ST38', 'H131800734_and_ST38'): 90014,
 ('ST38', 'CFT073_and_ST38'): 104317,
 ('ST38', '063_STEC_and_ST38'): 103640,
 ('CFT073', 'CFT073_and_ST38'): 104317,
 ('CFT073', '063_STEC_and_CFT073'): 40251,
 ('CFT073', 'CFT073_and_H131800734'): 99509}

In [6]:
import pandas as pd
from collections import defaultdict

def get_nb_of_correct_probes_found(report_file):
    df = pd.read_csv(report_file, sep="\t")
    df["tp"] = df["classification"].apply(
        lambda classification: classification in ["primary_correct", "secondary_correct", "supplementary_correct"]
    )
    return df["tp"].sum()

all_reports = glob.glob("/home/leandro/git/pandora1_paper/snippy_debug_2/hps/nobackup/research/zi/leandro/pandora1_paper/analysis_output_new_recall_pandora_nanopore_100x_best_snippy_precision_flank_150_recall_flank_50/recall/reports/**/*.tsv", recursive=True)

tool_to_samples_to_recall = defaultdict(defaultdict)
for report_file in all_reports:
    nb_of_correct_probes_found = get_nb_of_correct_probes_found(report_file)
    
    report_file_split = report_file.split("/")
    sample = report_file_split[report_file_split.index("reports")+1]
    tool = report_file_split[report_file_split.index("reports")+3]
    comparison = report_file_split[-1].replace(".report.tsv", "")
    
    recall = round(nb_of_correct_probes_found / samples_to_nb_of_truth_probes[(sample, comparison)] * 100, 1)
    
    tool_to_samples_to_recall[tool][(sample, comparison)] = recall
tool_to_samples_to_recall

defaultdict(collections.defaultdict,
            {'pandora_nanopore_with_denovo': defaultdict(None,
                         {('063_STEC', '063_STEC_and_H131800734'): 91.0,
                          ('063_STEC', '063_STEC_and_ST38'): 89.8,
                          ('063_STEC', '063_STEC_and_CFT073'): 85.7,
                          ('H131800734', '063_STEC_and_H131800734'): 91.6,
                          ('H131800734', 'CFT073_and_H131800734'): 91.2,
                          ('H131800734', 'H131800734_and_ST38'): 90.1,
                          ('ST38', '063_STEC_and_ST38'): 90.1,
                          ('ST38', 'H131800734_and_ST38'): 90.2,
                          ('ST38', 'CFT073_and_ST38'): 90.1,
                          ('CFT073', 'CFT073_and_H131800734'): 92.3,
                          ('CFT073', '063_STEC_and_CFT073'): 89.7,
                          ('CFT073', 'CFT073_and_ST38'): 91.7}),
             'snippy_NZ_CP016497.1': defaultdict(None,
                         {(

In [10]:
for sample_the_probe_is_from, comparison in tool_to_samples_to_recall["pandora_nanopore_with_denovo"]:
    print(f"Probes from {sample_the_probe_is_from}, comparison is {comparison}")
    
    if "063_STEC" in comparison or "CFT073" in comparison:
        better_tool = "snippy"
    else:
        better_tool = "pandora"
    
    pandora_recall = tool_to_samples_to_recall["pandora_nanopore_with_denovo"][(sample_the_probe_is_from, comparison)]
    snippy_recall = tool_to_samples_to_recall["snippy_NZ_CP016497.1"][(sample_the_probe_is_from, comparison)]
    difference = round(abs(pandora_recall-snippy_recall), 1)
    if better_tool == "snippy":
        if snippy_recall > pandora_recall:
            print(f"Snippy is predicted to be better and it is: {snippy_recall} vs {pandora_recall}, diff = {difference}")
        else:
            print(f"Snippy is predicted to be better and IT IS NOT: {snippy_recall} vs {pandora_recall}, diff = {difference}")
    else:
        if pandora_recall > snippy_recall:
            print(f"Pandora is predicted to be better and it is: {pandora_recall} vs {snippy_recall}, diff = {difference}")
        else:
            print(f"Pandora is predicted to be better and IT IS NOT: {pandora_recall} vs {snippy_recall}, diff = {difference}")
    print("")

Probes from 063_STEC, comparison is 063_STEC_and_H131800734
Snippy is predicted to be better and it is: 97.1 vs 91.0, diff = 6.1

Probes from 063_STEC, comparison is 063_STEC_and_ST38
Snippy is predicted to be better and it is: 94.9 vs 89.8, diff = 5.1

Probes from 063_STEC, comparison is 063_STEC_and_CFT073
Snippy is predicted to be better and it is: 91.8 vs 85.7, diff = 6.1

Probes from H131800734, comparison is 063_STEC_and_H131800734
Snippy is predicted to be better and it is: 94.2 vs 91.6, diff = 2.6

Probes from H131800734, comparison is CFT073_and_H131800734
Snippy is predicted to be better and it is: 94.3 vs 91.2, diff = 3.1

Probes from H131800734, comparison is H131800734_and_ST38
Pandora is predicted to be better and it is: 90.1 vs 87.0, diff = 3.1

Probes from ST38, comparison is 063_STEC_and_ST38
Snippy is predicted to be better and it is: 90.6 vs 90.1, diff = 0.5

Probes from ST38, comparison is H131800734_and_ST38
Pandora is predicted to be better and it is: 90.2 vs 86.3